**Efficient Yelp API Calls (Core)**

**Author: P. Slabber**

For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

In [1]:
# Standard Importsimport numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json, os, math, time
# Additional Importsimport os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/tspiet/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Steak'

In [4]:
# Specifying JSON_FILE filename (can include a folder)# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_Steak.json"
JSON_FILE

'Data/results_in_progress_NY_Steak.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: if file_exists == False:
    
## CREATE ANY NEEDED FOLDERS# Get the Folder Name only
folder = os.path.dirname(JSON_FILE)
## If JSON_FILE included a folder:if len(folder)>0:
# create the folder
os.makedirs(folder,exist_ok=True)
        
        
## INFORM USER AND SAVE EMPTY LISTprint(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
# save an empty list
with open(JSON_FILE,'w') as f:
    json.dump([],f)  

# If it exists, inform userelse:
print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_Steak.json already exists.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results

17100

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

855

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [13]:
def create_json_file(JSON_FILE, delete_if_exists=False):
    # Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    # If it DOES exist:
    if file_exists == True:
        # Check if user wants to delete if exists
        if delete_if_exists:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            # delete file
            os.remove(JSON_FILE)
            # Recursive call to function after old file deleted
            create_json_file(JSON_FILE, delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")
    
    # If it does NOT exist:
    else:
        # INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")

        # CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)

        # If JSON_FILE included a folder:
        if len(folder) > 0:
            # create the folder
            os.makedirs(folder, exist_ok=True)
        
        # Save empty list to start the json file
        with open(JSON_FILE, 'w') as f:
            json.dump([], f)

# Example usage:
# create_json_file("example.json", delete_if_exists=True)

In [14]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_NY_Steak.json already exists. Deleting previous file...
[i] Data/results_in_progress_NY_Steak.json not found. Saving empty list to new file.
- 0 previous results found.


815

In [15]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

# Exceeded 1000 api calls. Stopping loop.

  0%|          | 0/815 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [16]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,5CuYWr0-cE01HxlBu5Ka6g,carne-mare-new-york-5,Carne Mare,https://s3-media4.fl.yelpcdn.com/bphoto/fwpce8...,False,https://www.yelp.com/biz/carne-mare-new-york-5...,241,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.5,"{'latitude': 40.70568649199326, 'longitude': -...",[delivery],"{'address1': '89 South St', 'address2': '', 'a...",+12122804600,(212) 280-4600,666.157802,NaN
1,8XHbZIKDQbXhyKIGiukVeQ,boucherie-union-square-new-york-2,Boucherie Union Square,https://s3-media4.fl.yelpcdn.com/bphoto/WqE28s...,False,https://www.yelp.com/biz/boucherie-union-squar...,1446,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.5,"{'latitude': 40.73725824850519, 'longitude': -...","[pickup, delivery]","{'address1': '225 Park Ave S', 'address2': Non...",+12123530200,(212) 353-0200,3574.252680,$$$
2,DGhWO1sUWydVeR5j5ZZaMw,la-grande-boucherie-new-york-2,La Grande Boucherie,https://s3-media3.fl.yelpcdn.com/bphoto/b9URGc...,False,https://www.yelp.com/biz/la-grande-boucherie-n...,2563,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.5,"{'latitude': 40.7626274, 'longitude': -73.9808...","[pickup, delivery]","{'address1': '145 W 53rd St', 'address2': '', ...",+12125107714,(212) 510-7714,6458.633758,$$$
3,V3dHB6xAdd8_dH7t481PAw,steak-frites-bistro-new-york-3,Steak Frites Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/EiqUM4...,False,https://www.yelp.com/biz/steak-frites-bistro-n...,116,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.5,"{'latitude': 40.75559939381978, 'longitude': -...",[],"{'address1': '496 9th Ave', 'address2': None, ...",+19296145070,(929) 614-5070,5576.932575,$$$
4,hiTfA7vnxDSLv77nl6UQXQ,st-anselm-brooklyn,St. Anselm,https://s3-media4.fl.yelpcdn.com/bphoto/7-28AN...,False,https://www.yelp.com/biz/st-anselm-brooklyn?ad...,967,"[{'alias': 'newamerican', 'title': 'New Americ...",4.0,"{'latitude': 40.7143299, 'longitude': -73.95604}","[pickup, delivery]","{'address1': '355 Metropolitan Ave', 'address2...",+17183845054,(718) 384-5054,3369.159469,$$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
995,6ur9R2Fmr-7qXlNnyoclEg,blume-hütte-new-york,Blume/Hütte,https://s3-media2.fl.yelpcdn.com/bphoto/r9EcfF...,False,https://www.yelp.com/biz/blume-h%C3%BCtte-new-...,52,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 40.77754, 'longitude': -73.9517855}",[delivery],"{'address1': '1652 2nd Ave', 'address2': '', '...",+16469810764,(646) 981-0764,8780.141295,$$$
996,Twwpqrl15EVhmaNnNSrSQw,t-fusion-steakhouse-brooklyn,T Fusion Steakhouse,https://s3-media3.fl.yelpcdn.com/bphoto/sbEc7r...,False,https://www.yelp.com/biz/t-fusion-steakhouse-b...,95,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.0,"{'latitude': 40.61336, 'longitude': -73.93929}","[pickup, delivery]","{'address1': '3223 Quentin Rd', 'address2': ''...",+17189980002,(718) 998-0002,11245.591058,$$$
997,vXYFa2qzppkGiOiVPrLwDQ,don-peppe-south-ozone-park-2,Don Peppe,https://s3-media1.fl.yelpcdn.com/bphoto/-9zuAl...,False,https://www.yelp.com/biz/don-peppe-south-ozone...,698,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.6689814908178, 'longitude': -7...",[delivery],"{'address1': '135-58 Lefferts Blvd', 'address2...",+17188457587,(718) 845-7587,15112.179354,$$$
998,-BOAKHyKKAXE1WhuVpMFkQ,the-good-fork-pub-brooklyn-2,The Good Fork Pub,https://s3-media1.fl.yelpcdn.com/bphoto/qwmYaX...,False,https://www.yelp.com/biz/the-good-fork-pub-bro...,415,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.0,"{'latitude': 40.67599, 'longitude': -74.01432}",[],"{'address1': '391 Van Brunt St', 'address2': '...",+17186436636,(718) 643-6636,3685.119835,$$
999,5u4T6-Z-KhHrRZ9g54pugQ,carols-bun-new-york-3,Carol's Bun,https://s3-media1.fl.yelpcdn.com/bphoto/igKwef...,False,https://www.yelp.com/biz/carols-bun-new-york-3...,64,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.5,"{'latitude': 40.7138317002555, 'longitude': -7...","[pickup, delivery]","{'address1': '139 E Broadway', 'address2': '',...",+12125660203,(212) 566-0203,962.758314,$


In [17]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_Steak.csv.gz', compression='gzip',index=False)